# 대회 데이터 pre-processing 코드 

- 대회 데이터를 scale 에 따라서 split 하고 저장 

- 00.Data 폴더 아래에 oiltank_dataset/roas 

- 아래와 같이 폴더 생성   
 roas_split_1024_gap_128_x1  
 roas_split_1024_gap_128_x1.5  
 roas_split_1024_gap_128_x2 

In [9]:
# 주요 라이브러리 로드 

#import utils as util
import os
#import imgsplit_multiprocess
#import split_onlyimage_multiprocess
import shutil
from multiprocessing import Pool
#from geojson2coco import geojson2coco
import argparse

In [10]:
# imgsplit_multiprocess.splitbase 

import os
import codecs
import math
import json
import copy
from multiprocessing import Pool
from functools import partial
import time

import numpy as np
import cv2
import shapely.geometry as shgeo


In [11]:
# util function들 로드 

from split_utils import *

In [12]:
# 데이터 전처리를 위한 함수코드 

def get_coco_template():
    geometry = {'coordinates':[[[1, 1, 0.0],
                               [1, 1, 0.0],
                               [1, 1, 0.0],
                               [1, 1, 0.0]]],
                'type': 'Polygon'},
    properties = {
        'object_imcoords': '',
        'object_angle': 0, #=> 0이 아니고, parsing이 필요하다. 
        'building_imcoords': 'EMPTY',
        'road_imcoords': 'EMPTY',
        'image_id': '',
        'original_image_id': '',
        'ingest_time': '',
        'type_id': '',
        'type_name': ''
    }
    return dict(geometry=geometry, properties=properties, type='Feature')


def parse_nia_poly(filename):
    """
        parse the dota ground truth in the format:
        [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
    """
    objects = []
    with open(filename, 'r') as f:
        label_info = json.load(f)

    for obj in label_info['features']:
        properties = obj['properties']
        object_struct = dict()
        object_struct['original_image_id'] = properties['image_id']
        object_struct['ingest_time'] = properties['ingest_time']
        object_struct['type_id'] = 1 # eric edit 1
        object_struct['type_name'] = properties['type_name']


        polys = list(map(float, properties['object_imcoords'].split(',')))
        object_struct['poly'] = list(map(int, polys))

        tuple_poly = [tuple(polys[i:i+2]) for i in range(0,8,2)]
        gtpoly = shgeo.Polygon(tuple_poly)
        object_struct['area'] = gtpoly.area

        objects.append(object_struct)

    return objects


def choose_best_pointorder_fit_another(poly1, poly2):
    """
        To make the two polygons best fit with each point
    """
    x1 = poly1[0]
    y1 = poly1[1]
    x2 = poly1[2]
    y2 = poly1[3]
    x3 = poly1[4]
    y3 = poly1[5]
    x4 = poly1[6]
    y4 = poly1[7]
    combinate = [np.array([x1, y1, x2, y2, x3, y3, x4, y4]), np.array([x2, y2, x3, y3, x4, y4, x1, y1]),
                 np.array([x3, y3, x4, y4, x1, y1, x2, y2]), np.array([x4, y4, x1, y1, x2, y2, x3, y3])]
    dst_coordinate = np.array(poly2)
    distances = np.array([np.sum((coord - dst_coordinate)**2) for coord in combinate])
    sorted = distances.argsort()
    return combinate[sorted[0]]

def cal_line_length(point1, point2):
    return math.sqrt( math.pow(point1[0] - point2[0], 2) + math.pow(point1[1] - point2[1], 2))



class splitbase():
    def __init__(self,
                 basepath,
                 outpath,
                 code = 'utf-8',
                 gap=128,
                 subsize=1024,
                 thresh=0.7, # default 0.7
                 choosebestpoint=True,
                 ext = '.png',
                 padding=True,
                 num_process=8
                 ):
        """
        :param basepath: base path for dota data
        :param outpath: output base path for dota data,
        the basepath and outputpath have the similar subdirectory, 'images' and 'labelTxt'
        :param code: encodeing format of txt file
        :param gap: overlap between two patches
        :param subsize: subsize of patch
        :param thresh: the thresh determine whether to keep the instance if the instance is cut down in the process of split
        :param choosebestpoint: used to choose the first point for the
        :param ext: ext for the image format
        :param padding: if to padding the images so that all the images have the same size
        """
        self.basepath = basepath
        self.outpath = outpath
        self.code = code
        self.gap = gap
        self.subsize = subsize
        self.slide = self.subsize - self.gap
        self.thresh = thresh
        self.imagepath = os.path.join(self.basepath, 'images')
        self.labelpath = os.path.join(self.basepath, 'json')
        self.outimagepath = os.path.join(self.outpath, 'images')
        self.outlabelpath = os.path.join(self.outpath, 'json')
        self.choosebestpoint = choosebestpoint
        self.ext = ext
        self.padding = padding
        self.pool = Pool(num_process)
        print('padding:', padding)

        if not os.path.isdir(self.outpath):
            os.mkdir(self.outpath)
        if not os.path.isdir(self.outimagepath):
            os.mkdir(self.outimagepath)
        if not os.path.isdir(self.outlabelpath):
            os.mkdir(self.outlabelpath)
    ## point: (x, y), rec: (xmin, ymin, xmax, ymax)
    # def __del__(self):
    #     self.f_sub.close()
    ## grid --> (x, y) position of grids
    def polyorig2sub(self, left, up, poly):
        polyInsub = np.zeros(len(poly))
        for i in range(int(len(poly)/2)):
            polyInsub[i * 2] = int(poly[i * 2] - left)
            polyInsub[i * 2 + 1] = int(poly[i * 2 + 1] - up)
        return polyInsub

    def calchalf_iou(self, poly1, poly2):
        """
            It is not the iou on usual, the iou is the value of intersection over poly1
        """
        inter_poly = poly1.intersection(poly2)
        inter_area = inter_poly.area
        poly1_area = poly1.area
        half_iou = inter_area / poly1_area
        return inter_poly, half_iou

    def saveimagepatches(self, img, subimgname, left, up):
        subimg = copy.deepcopy(img[up: (up + self.subsize), left: (left + self.subsize)])
        outdir = os.path.join(self.outimagepath, subimgname + self.ext)
        h, w, c = np.shape(subimg)
        if (self.padding):
            outimg = np.zeros((self.subsize, self.subsize, 3))
            outimg[0:h, 0:w, :] = subimg
            cv2.imwrite(outdir, outimg)
        else:
            cv2.imwrite(outdir, subimg)

    def GetPoly4FromPoly5(self, poly):
        distances = [cal_line_length((poly[i * 2], poly[i * 2 + 1] ), (poly[(i + 1) * 2], poly[(i + 1) * 2 + 1])) for i in range(int(len(poly)/2 - 1))]
        distances.append(cal_line_length((poly[0], poly[1]), (poly[8], poly[9])))
        pos = np.array(distances).argsort()[0]
        count = 0
        outpoly = []
        while count < 5:
            #print('count:', count)
            if (count == pos):
                outpoly.append((poly[count * 2] + poly[(count * 2 + 2)%10])/2)
                outpoly.append((poly[(count * 2 + 1)%10] + poly[(count * 2 + 3)%10])/2)
                count = count + 1
            elif (count == (pos + 1)%5):
                count = count + 1
                continue

            else:
                outpoly.append(poly[count * 2])
                outpoly.append(poly[count * 2 + 1])
                count = count + 1
        return outpoly

    def savepatches(self, resizeimg, objects, subimgname, left, up, right, down):
        outdir = os.path.join(self.outlabelpath, subimgname + '.json')
        mask_poly = []
        imgpoly = shgeo.Polygon([(left, up), (right, up), (right, down),
                                 (left, down)])
        features = []
        for obj in objects:
            gtpoly = shgeo.Polygon([(obj['poly'][0], obj['poly'][1]),
                                        (obj['poly'][2], obj['poly'][3]),
                                        (obj['poly'][4], obj['poly'][5]),
                                        (obj['poly'][6], obj['poly'][7])])
            if (gtpoly.area <= 0):
                continue
            inter_poly, half_iou = self.calchalf_iou(gtpoly, imgpoly)

            # print('writing...')
            if (half_iou == 1):
                polyInsub = self.polyorig2sub(left, up, obj['poly'])
                coco_obj = get_coco_template()
                coco_obj['properties']['object_imcoords'] = ','.join(list(map(str, polyInsub)))
                #--- edit by eric 
                # print("#"*10)
                # print("obj",obj)
                # print(obj['object_angle'])
                # coco_obj['properties']['object_angle'] = obj['object_angle']
                #---
                coco_obj['properties']['original_image_id'] = obj['original_image_id']
                coco_obj['properties']['image_id'] = subimgname+'.png'
                coco_obj['properties']['ingest_time'] = obj['ingest_time']
                coco_obj['properties']['type_id'] = 1
                coco_obj['properties']['type_name'] = obj['type_name']

                features.append(coco_obj)

                #outline = ' '.join(list(map(str, polyInsub)))
                #outline = outline + ' ' + obj['name'] + ' ' + str(obj['difficult'])
                #f_out.write(outline + '\n')
            elif (half_iou > 0):
            #elif (half_iou > self.thresh):
                ##  print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
                inter_poly = shgeo.polygon.orient(inter_poly, sign=1)
                out_poly = list(inter_poly.exterior.coords)[0: -1]
                if len(out_poly) < 4:
                    continue

                out_poly2 = []
                for i in range(len(out_poly)):
                    out_poly2.append(out_poly[i][0])
                    out_poly2.append(out_poly[i][1])

                if (len(out_poly) == 5):
                    #print('==========================')
                    out_poly2 = self.GetPoly4FromPoly5(out_poly2)
                elif (len(out_poly) > 5):
                    """
                        if the cut instance is a polygon with points more than 5, we do not handle it currently
                    """
                    continue
                if (self.choosebestpoint):
                    out_poly2 = choose_best_pointorder_fit_another(out_poly2, obj['poly'])

                polyInsub = self.polyorig2sub(left, up, out_poly2)

                for index, item in enumerate(polyInsub):
                    if (item <= 1):
                        polyInsub[index] = 1
                    elif (item >= self.subsize):
                        polyInsub[index] = self.subsize

                if (half_iou > self.thresh):
                    coco_obj = get_coco_template()
                    coco_obj['properties']['object_imcoords'] = ','.join(list(map(str, polyInsub)))
                    coco_obj['properties']['original_image_id'] = obj['original_image_id']
                    coco_obj['properties']['image_id'] = subimgname+'.png'
                    coco_obj['properties']['ingest_time'] = obj['ingest_time']
                    coco_obj['properties']['type_id'] = obj['type_id']
                    coco_obj['properties']['type_name'] = obj['type_name']
                    #--- edit by eric 
                    #coco_obj['properties']['object_angle'] = obj['object_angle']

                    features.append(coco_obj)

                #outline = ' '.join(list(map(str, polyInsub)))
                #if (half_iou > self.thresh):
                #    outline = outline + ' ' + obj['name'] + ' ' + str(obj['difficult'])
                #else:
                #    ## if the left part is too small, label as '2'
                #    outline = outline + ' ' + obj['name'] + ' ' + '2'
                #f_out.write(outline + '\n')
            #else:
                #   mask_poly.append(inter_poly)
        with open(outdir, 'w') as f_out:
            json.dump(dict(features=features, type='FeatureCollection'), f_out)
        self.saveimagepatches(resizeimg, subimgname, left, up)

    def SplitSingle(self, name, rate, extent):
        """
            split a single image and ground truth
        :param name: image name
        :param rate: the resize scale for the image
        :param extent: the image format
        :return:
        """
        try:
            img = cv2.imread(os.path.join(self.imagepath, name + extent))
            print('img name:', name)
        except:
            print('img name:', name)
        if np.shape(img) == ():
            return
        fullname = os.path.join(self.labelpath, name + '.json')
        print("fullname : ",fullname)
        objects = parse_nia_poly(fullname)
        for obj in objects:
            obj['poly'] = list(map(lambda x:rate*x, obj['poly']))

        if (rate != 1):
            resizeimg = cv2.resize(img, None, fx=rate, fy=rate, interpolation = cv2.INTER_CUBIC)
        else:
            resizeimg = img

        outbasename = name + '__' + str(rate) + '__'
        weight = np.shape(resizeimg)[1]
        height = np.shape(resizeimg)[0]

        # if (max(weight, height) < self.subsize):
        #     return

        left, up = 0, 0
        while (left < weight):
            if (left + self.subsize >= weight):
                left = max(weight - self.subsize, 0)
            up = 0
            while (up < height):
                if (up + self.subsize >= height):
                    up = max(height - self.subsize, 0)
                right = min(left + self.subsize, weight - 1)
                down = min(up + self.subsize, height - 1)
                subimgname = outbasename + str(left) + '___' + str(up)
                # self.f_sub.write(name + ' ' + subimgname + ' ' + str(left) + ' ' + str(up) + '\n')
                self.savepatches(resizeimg, objects, subimgname, left, up, right, down)
                if (up + self.subsize >= height):
                    break
                else:
                    up = up + self.slide
            if (left + self.subsize >= weight):
                break
            else:
                left = left + self.slide

    def splitdata(self, rate):
        """
        :param rate: resize rate before cut
        """
        print("img path : ", self.imagepath)
        imagelist = GetFileFromThisRootDir(self.imagepath)
        print("imagelist : " ,imagelist)
        imagenames = [get_basename(x) for x in imagelist if (get_basename(x) != 'Thumbs')]
        print('name:',imagenames)

        worker = partial(self.SplitSingle, rate=rate, extent=self.ext)
        #
        # for name in imagenames:
        #     self.SplitSingle(name, rate, self.ext)
        self.pool.map(worker, imagenames)

    def __getstate__(self):
        self_dict = self.__dict__.copy()
        del self_dict['pool']
        return self_dict

    def __setstate__(self, state):
        self.__dict__.update(state)

In [13]:
def parse_args(roas_path,split_gap,rate):
    parser = argparse.ArgumentParser(description='prepare roas_for_split')
    parser.add_argument('--srcpath', default=roas_path)
    parser.add_argument('--dstpath', default=os.path.join(roas_path,f'roas_split_1024_gap_{split_gap}_x{rate}'),
                        help='prepare data')
    args = parser.parse_args([])

    return args

In [14]:
def run_split(roas_root):

    # 3가지 스케일로 이미지 처리  
    rates = [1,1.5,2]
    for rate in rates:
        train_dst_name = f'train1024_{rate}x'
        test_dst_name = f'test1024_{rate}x' #'test1024_2x'
        split_gap = 128

        args = parse_args(roas_path=roas_root,split_gap=split_gap,rate=rate)
        srcpath = args.srcpath
        dstpath = args.dstpath

        # prepare function 
        if not os.path.exists(os.path.join(dstpath, test_dst_name)):
            os.makedirs(os.path.join(dstpath, test_dst_name))
        if not os.path.exists(os.path.join(dstpath, train_dst_name)):
            os.makedirs(os.path.join(dstpath, train_dst_name))
        
        # split object 
        split_train = splitbase(os.path.join(srcpath, 'train'),
                            os.path.join(dstpath, train_dst_name),
                            gap=split_gap, # 
                            subsize=1024,
                            num_process=32
                            )
        
        # split exec
        split_train.splitdata(rate)

In [15]:
# 함수실행 
# 실행후에 아래의 00.Data/oiltank_dataset/roas 폴더안에 
# roas_split_1024_gap_128_x1
# roas_split_1024_gap_128_x1.5
# roas_split_1024_gap_128_x2 
# 3가지 폴더가 생성 및 폴더 하위에 scale 조정된 이미지 파일들 위치


run_split(roas_root = "/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas")

padding: True
img path :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images
imagelist :  ['/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ05252_PS3_K3A_NIA0341.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ03621_PS3_K3A_NIA0151.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ03166_PS3_K3_NIA0128.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ03513_PS3_K3A_NIA0146.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ05624_PS3_K3A_NIA0358.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ05003_PS3_K3A_NIA0328.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ04514_PS3_K3A_NIA0298.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ03514_PS3_K3A_NIA0146.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ03468_PS3_K3A_NIA0141.png', '/mnt/hdd/eric/.tmp_ipy

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


padding: True
img path :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images
imagelist :  ['/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ05252_PS3_K3A_NIA0341.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ03621_PS3_K3A_NIA0151.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ03166_PS3_K3_NIA0128.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ03513_PS3_K3A_NIA0146.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ05624_PS3_K3A_NIA0358.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ05003_PS3_K3A_NIA0328.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ04514_PS3_K3A_NIA0298.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ03514_PS3_K3A_NIA0146.png', '/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/images/OBJ03468_PS3_K3A_NIA0141.png', '/mnt/hdd/eric/.tmp_ipy

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8

img name: OBJ04367_PS3_K3A_NIA0285
fullname :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04367_PS3_K3A_NIA0285.json
img name:

...



  

OBJ05003_PS3_K3A_NIA0328
/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04434_PS3_K3A_NIA0291.json  fullname : fullname : /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ03166_PS3_K3_NIA0128.json /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ03973_PS3_K3A_NIA0166.json /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ03785_PS3_K3A_NIA0158.json 

/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ05760_PS3_K3A_NIA0365.json
 /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ03807_PS3_K3A_NIA0159.json fullname : /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04560_PS3_K3A_NIA0300.json/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ03728_PS3_K3A_NIA0156.json
fullname :   fullname : 
/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ05273_PS3_K3A_NIA0342.json
/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ03810_

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8

img name: OBJ04367_PS3_K3A_NIA0285
fullname :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04367_PS3_K3A_NIA0285.jsonimg name:
 img name:OBJ05069_PS3_K3A_NIA0330 
OBJ04673_PS3_K3A_NIA0312fullname :  
/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ05069_PS3_K3A_NIA0330.json
fullname :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04673_PS3_K3A_NIA0312.json
img name: OBJ03563_PS3_K3A_NIA0150
fullname :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ03563_PS3_K3A_NIA0150.json
img name: OBJ04702_PS3_K3A_NIA0313
fullname :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04702_PS3_K3A_NIA0313.json
img name: OBJ05300_PS3_K3A_NIA0344
fullname :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ05300_PS3_K3A_NIA0344.json
img name: OBJ04640_PS3_K3A_NIA0305
fullname :  img name:img name:/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04640_PS3_K3A_NIA0305.json 
 

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


img name:img name:  OBJ04007_PS3_K3A_NIA0167OBJ04792_PS3_K3A_NIA0297
fullname : 
 fullname : /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04007_PS3_K3A_NIA0167.json 
/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04792_PS3_K3A_NIA0297.json
img name: OBJ04750_PS3_K3A_NIA0314
fullname :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04750_PS3_K3A_NIA0314.json
img name: OBJ03522_PS3_K3A_NIA0147
fullname :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ03522_PS3_K3A_NIA0147.json
img name: OBJ04561_PS3_K3A_NIA0300
fullname :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ04561_PS3_K3A_NIA0300.json
img name: OBJ05001_PS3_K3A_NIA0328
fullname :  /mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ05001_PS3_K3A_NIA0328.json
img name: OBJ03550_PS3_K3A_NIA0149
img name:fullname :   OBJ04912_PS3_K3A_NIA0322/mnt/hdd/eric/.tmp_ipy/00.Data/oiltank_dataset/roas/train/json/OBJ03550_PS3_K3A